In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

Looking in indexes: https://download.pytorch.org/whl/cu128
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T

In [18]:
mnist_data_train = datasets.MNIST(root="./root", train=True, download=True, transform=T.ToTensor())
mnist_data_test = datasets.MNIST(root="./root", train=False, download=True, transform=T.ToTensor())

In [19]:
train_dataloader = DataLoader(mnist_data_train, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
test_dataloader = DataLoader(mnist_data_test, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)

In [36]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(28*28, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, 10)
        self.activation = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.flatten(x)
        x = self.layer1(x)
        x = self.activation(x)
        x = self.layer2(x)
        x = self.activation(x)
        x = self.layer3(x)
        x= self.activation(x)
        return x

In [37]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else 'cpu'
device

device(type='cuda')

In [38]:
model = NeuralNetwork().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [39]:
def train(train_loader : DataLoader, model: nn.Module, criterion : nn.Module, optimizer : torch.optim):
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = criterion(y_pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        

In [40]:
train(train_dataloader, model, criterion, optimizer)

In [20]:
torch.Tensor([1]).grad

In [8]:
np.array([1])

array([1])